In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
mark = 'tuwen'
info_dir = f'/search/odin/publicData/CloudS/rank/infos/{mark}/8'
hour = 2020033122
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df.head()

/search/odin/publicData/CloudS/rank/infos/tuwen/8/2020033122/valid.csv


,doc_topic,abtestid,article_page_time,did,distribution,docid,duration,id,impression_time,lr_score,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,17,1585481474,-6132433720895875250,3057,open_doc_prod24573924,0,012d864427038290052\topen_doc_prod24573924,1585664976,0.250715,...,0,0,1.0,0,sgsapp,0.085941,0.477194,0.118615,2020033122,0
1,0,12,1585651918,-199435366238330142,3039,open_doc_prod24845962,0,b7c8863515046574458\topen_doc_prod24845962,1585664457,0.159024,...,0,0,1.0,0,sgsapp,0.051096,0.211403,0.255982,2020033122,0
2,0,18,1584994781,8379849512726347899,0,2032440l4kwHSb,0,8670b262e99e6375198d85170a51cf63f9fa5b285c89\t...,1585664077,0.253672,...,0,0,1.0,0,sgsapp,0.019233,0.146144,0.125822,2020033122,0
3,0,2,1585568084,5652414863303639373,3039,open_doc_prod24699810,0,107e866089035724943\topen_doc_prod24699810,1585664588,0.134533,...,0,0,1.0,0,sgsapp,0.011056,0.078550,0.180714,2020033122,0
4,0,2,1585618282,-6478976331430339996,0,2033190l2FcAno,0,a7b1927b4f2cee73a40399d3d7ee76a09d7ed7d7cd51\t...,1585664661,0.442333,...,0,0,1.0,0,sgsapp,0.093484,0.447667,0.150609,2020033122,0


In [4]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 1780/1780 [01:20<00:00, 22.10it/s]


In [17]:
df['click'] = (df.dur > 0).astype(int)

In [5]:
pd.set_option('display.max_rows', 1000)

In [7]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [18]:
len(df)

7324548

In [19]:
mean_duration = df[df.dur > 0].dur.mean()

In [20]:
mean_duration

91.64048924859095

In [21]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [22]:
cold_mids = set(df[df.cold==True].mid.values)

In [23]:
len(set(df.mid.values))

379333

In [24]:
len(cold_mids)

39932

In [25]:
d8 = df[df.abtestid==8]

In [27]:
d456 = df[df.abtestid.isin([4,5,6])]

In [30]:
len(d8)

369649

In [31]:
len(d456)

1100906

In [35]:
d8[['position', 'click']]

,position,click
27,6,0
28,2,0
38,6,0
118,1,0
151,4,0
...,...,...
7324432,8,0
7324451,12,0
7324465,12,0
7324484,5,0


In [37]:
d8[['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.371429
0,0.244958
1,0.173736
2,0.150259
3,0.136885
4,0.138395
5,0.114407
6,0.117048
7,0.104843


In [38]:
d456[['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.255411
0,0.237517
1,0.167314
2,0.145679
3,0.134502
4,0.133117
5,0.112512
6,0.114123
7,0.101481


In [40]:
len(d456[d456.position==0])

124366

In [41]:
d8[d8.cold==1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.833333
0,0.070632
1,0.049369
2,0.036918
3,0.011494
4,0.020000
5,0.033333
6,0.026432
7,0.035714


In [42]:
len(d8[d8.cold==1])

10947

In [43]:
d456[d456.cold==1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.375000
0,0.066792
1,0.045658
2,0.038681
3,0.057508
4,0.036765
5,0.075000
6,0.024806
7,0.029268


In [44]:
d8.columns

Index(['doc_topic', 'abtestid', 'article_page_time', 'did', 'distribution',
       'docid', 'duration', 'id', 'impression_time', 'lr_score', 'mid',
       'mobile_brand', 'mobile_model', 'mobile_screen_width', 'model_name',
       'model_ver', 'network', 'num_interests', 'ori_lr_score', 'position',
       'rea', 'read_completion_rate', 'show_time', 'time_interval',
       'time_weekday', 'timespan_interval', 'today_refresh_num', 'type', 'uid',
       'unlike', 'video_time', 'click', 'weight', 'cold', 'product_data',
       'pred', 'pred_click', 'pred_dur', 'hour', 'dur', 'title', 'page_time'],
      dtype='object')

In [46]:
set(d8.network)

{0, 1, 3, 4, 5}

In [47]:
d8[d8.network==1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.389831
0,0.244971
1,0.173034
2,0.151056
3,0.138026
4,0.138537
5,0.114143
6,0.116569
7,0.103828


In [52]:
d8[d8.network!=1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.272727
0,0.244878
1,0.178351
2,0.145535
3,0.127881
4,0.137336
5,0.116428
6,0.121059
7,0.113811


In [53]:
d456[d456.network==1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.305732
0,0.238526
1,0.168205
2,0.147797
3,0.136570
4,0.133507
5,0.112225
6,0.114316
7,0.100153


In [54]:
d456[d456.network!=1][['position', 'click']].groupby(['position']).mean()

,click
position,
-1,0.148649
0,0.231628
1,0.161502
2,0.133442
3,0.117632
4,0.130112
5,0.114764
6,0.112473
7,0.113685


In [55]:
d456[d456.network!=1].click.mean()

0.13707009938593964

In [56]:
d456[d456.network==1].click.mean()

0.13170892734289918

In [58]:
d456[d456.network==1].click.sum() / len(set(d456[d456.network==1].mid))

2.6034353084663073

In [59]:
d456[d456.network!=1].click.sum() / len(set(d456[d456.network!=1].mid))

2.2620087336244543

In [60]:
d8[d8.network==1].click.sum() / len(set(d8[d8.network==1].mid))

2.636336568808432

In [61]:
d8[d8.network!=1].click.sum() / len(set(d8[d8.network!=1].mid))

2.3512123438648054

In [62]:
d456[d456.network==1].click.sum() / len(set(d456[d456.network==1].mid))

2.6034353084663073

In [63]:
len(d456[d456.network==1])

972417

In [64]:
len(d456[d456.network!=1])

128489